# Predicting Consumer Disputing on the resolution provided by Bank

In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, fbeta_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

In [ ]:
cc_train=pd.read_csv(r'/content/Consumer_Complaints_train.csv')
cc_test=pd.read_csv(r'/content/Consumer_Complaints_test_share.csv')
cc_train['data']='train'
cc_test['data']='test'
cc_test['Consumer disputed?']=np.nan
cc_all=pd.concat([cc_train,cc_test],0,sort=False)
print(cc_train.isnull().sum())
col_name=['Date received','Date sent to company']

for i in col_name:
    cc_all[i]=pd.to_datetime(cc_all[i])

cc_all['Consumer disputed?']=(cc_all['Consumer disputed?']=='Yes').astype(int)

In [ ]:
cc_all.drop(['ZIP code','Complaint ID'],1,inplace=True)
cc_all['Date Diff']=(pd.to_datetime(cc_all['Date sent to company'])-pd.to_datetime(cc_all['Date received']))/np.timedelta64(1,'D')
def conversion(data,name):
    m=name+'_Month'
    d=name+'_Day'
    data[m]=data[name].dt.month
    data[d]=data[name].dt.day
    data[m+'_sin']=np.sin(2*np.pi*(data[m]/max(data[m])))
    data[m+'_cos']=np.cos(2*np.pi*(data[m]/max(data[m])))
    data[d+'_sin']=np.sin(2*np.pi*(data[d]/max(data[d])))
    data[d+'_cos']=np.cos(2*np.pi*(data[d]/max(data[d])))
    data=data.drop([m,d,name],1,inplace=True)
    return data

conversion(cc_all,'Date received')
conversion(cc_all,'Date sent to company')

for i in cc_all.select_dtypes(['object']).columns:
    cc_all[i]=np.where(cc_all[i].isnull(),'missing',cc_all[i])
    
    [(i,cc_all[i].isnull().sum()) for i in cc_all.columns]
    

In [ ]:
def dummies(data,col,n):
    freq=data[col].value_counts()
    k=freq.index[freq>n][:-1]
    for cat in k:
        name=col+"_"+str(cat)
        data[name]=(data[col]==cat).astype(int)
    del data[col]
    
dummies(cc_all,'Product',0)
dummies(cc_all,'Sub-product',5000)
dummies(cc_all,'Issue',5000)
dummies(cc_all,'Sub-issue',5000)
dummies(cc_all,'Company public response',5000)
dummies(cc_all,'Company',5000)
dummies(cc_all,'State',5000)
dummies(cc_all,'Tags',5000)
dummies(cc_all,'Consumer consent provided?',5000)
dummies(cc_all,'Timely response?',5000)
dummies(cc_all,'Submitted via',5000)
dummies(cc_all,'Company response to consumer',5000)

In [ ]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from string import punctuation
from nltk.tokenize import word_tokenize
lemma=WordNetLemmatizer()
from sklearn.feature_extraction.text import TfidfVectorizer
# import nltk
# nltk.download('stopwords')
# nltk.download('punkt')

In [ ]:
def split_into_lemmas(message):
    message=message.lower()
    for i in punctuation:
        message=message.replace(i,"")
    message=message.replace('x',"")
    words=word_tokenize(message)
    words_sans_stop=[]
    for word in words:
        if word in my_stop:continue
        if word.isdigit()==True:continue
        words_sans_stop.append(word)
    return [lemma.lemmatize(word) for word in words_sans_stop]


In [ ]:
tfidf=TfidfVectorizer(analyzer=split_into_lemmas,min_df=0.01,max_df=0.8,max_features=400,stop_words=my_stop)
tfidf.fit(cc_all['Consumer complaint narrative'])
ccn_data=tfidf.transform(cc_all['Consumer complaint narrative'].values.astype('U'))
ccn_data=pd.DataFrame(ccn_data.toarray(),columns=tfidf.get_feature_names())


In [ ]:
def report(result,n_top):
    for i in range(0,n_top+1):
        candidates=np.flatnonzero(result['rank_test_score']==i)
        for candidate in candidates:
            print('Model wiht Rank ',i)
            print("Mean Valdiation Score & Std Score",result['mean_test_score'][candidate],
                 result['std_test_score'][candidate])
            print("Parameter",result['params'][candidate])
            print("^_^")

In [ ]:
del cc_all['Consumer complaint narrative']
cc_all.reset_index(drop=True,inplace=True)
ccn_data.reset_index(drop=True,inplace=True)
cc=pd.concat([cc_all,ccn_data],1)
train=cc.loc[cc_all['data']=='train',]
del train['data']
test=cc.loc[cc_all['data']=='test',]
test=test.drop(['Consumer disputed?','data'],1)
model_train,val=train_test_split(train,test_size=0.4,random_state=2)
x_train=model_train.drop(['Consumer disputed?'],1)
y_train=model_train['Consumer disputed?']
x_val=val.drop(['Consumer disputed?'],1)
y_val=val['Consumer disputed?']

In [ ]:
log_param={'penalty':['l1','l2'],'C':[0.01,0.1,0.5,0.15,0.2],'class_weight':['balanced']}
clf=LogisticRegression()

log_grid=GridSearchCV(clf,cv=15,param_grid=log_param,scoring='roc_auc',n_jobs=-1,verbose=20)
log_grid.fit(x_train,y_train)

In [ ]:
report(log_grid.cv_results_,5)
log_grid.best_estimator_

In [ ]:
log=LogisticRegression(C=0.2, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l1',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)
log.fit(x_train,y_train)


In [ ]:
roc_auc_score(y_val,log.predict_proba(x_val)[:,1])
roc_auc_score(y_train,log.predict_proba(x_train)[:,1])


In [ ]:
test_score=log.predict_proba(test)[:,1]
cc_test['Consumer disputed?']=test_score